In [2]:
FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
DATOS_DIR   = '../Data_Sets/p2/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pylab as plt
from IPython import display
from ClassPerceptron import Perceptron
from sklearn import preprocessing
import grafica as gr
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# Regresión Logística multiclase
## Ejercicio 11

Se busca predecir si el tipo de fármaco que se debe administrar a un paciente afectado de rinitis alérgica es el habitual o no. Se dispone de información correspondiente a las historias clínicas de pacientes atendidos previamente


### a) 

Numerice los atributos nominales y utilice el archivo **drug_train.csv** para entrenar un modelo con 5 neuronas que sea capaz de predecir el tipo de fármaco que se debe administrar a un paciente afectado de rinitis alérgica.

In [3]:

data_train = pd.read_csv(DATOS_DIR + 'drugs_train.csv')

# Variables categóricas y numéricas
cat_cols = ["Sex", "BP", "Cholesterol"]
num_cols = ["Age", "Na", "K"]


# Preprocesador: OneHot a categóricas, passthrough a numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols)
    ]
)

# Separar X e y
X_train_raw = data_train.drop(columns=['Drug'])
y_train = data_train['Drug'].map({'drugA':0,'drugB':1,'drugC':2,'drugX':3,'drugY':4}).to_numpy()

# Aplicar preprocesamiento (fit en train)
X_train = preprocessor.fit_transform(X_train_raw)

# Entrenar tu modelo (ejemplo con regresión logística multinomial)

clf = LogisticRegression(max_iter=1000, multi_class="multinomial")
clf.fit(X_train, y_train)

# Accuracy en entrenamiento
y_pred_train = clf.predict(X_train)
acc_train = accuracy_score(y_train, y_pred_train)
print("Accuracy en entrenamiento:", acc_train)

Accuracy en entrenamiento: 0.60625


/home/manuel/Documents/Facultad/DeepLearning/env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### b) 

Luego utilice el archivo drug_test.csv para medir la calidad del modelo.

In [4]:
data_test = pd.read_csv(DATOS_DIR + 'drugs_test.csv')

X_test_raw = data_test.drop(columns=['Drug'])
y_test = data_test['Drug'].map({'drugA':0,'drugB':1,'drugC':2,'drugX':3,'drugY':4}).to_numpy()

# Transformar test con el preprocesador entrenado en train
X_test = preprocessor.transform(X_test_raw)

# Predicción en test
y_pred_test = clf.predict(X_test)
acc_test = accuracy_score(y_test, y_pred_test)
print("Accuracy en test:", acc_test)

Accuracy en test: 0.55
